In [11]:
import os

kaggle_path = '/kaggle/input/labeled-faces-in-the-wild-lfw-20180109/lfw'
default_path = '/data/raw'

if os.path.exists(kaggle_path):
    raw_dir = kaggle_path
else:
    raw_dir = default_path

print(f'Raw directory is set to: {raw_dir}')

Raw directory is set to: /kaggle/input/labeled-faces-in-the-wild-lfw-20180109/lfw


In [12]:
import random
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from PIL import Image
import itertools

class SiameseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_folder = datasets.ImageFolder(root=root_dir)
        self.transform = transform
        self.image_pairs = list(itertools.combinations_with_replacement(range(len(self.image_folder)), 2))
        self.targets = [int(self.image_folder.targets[idx1] == self.image_folder.targets[idx2]) for idx1, idx2 in self.image_pairs]

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, index):
        idx1, idx2 = self.image_pairs[index]
        img1,_ = self.image_folder[idx1]
        img2,_ = self.image_folder[idx2]
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, self.targets[index]

In [13]:
import torch
from torch.utils.data import Sampler
import numpy as np
from collections import Counter

class RandomUnderSampler(Sampler):
    def __init__ (self, targets, seed=None, shuffle=False):
        self.class_counts = Counter(targets)
        self.indices = {cls : np.where(self.targets==cls)[0] for cls in self.class_counts.keys()}
        self.seed = seed
        self.shuffle = shuffle

    def __iter__(self):
        sampled_indices = []
        for _, indices in self.indices.items():
            if self.seed is not None:
                np.random.seed(self.seed)
            sampled_indices.extend(np.random.choice(indices, self.__min_count()))
        if self.shuffle:
            np.random.shuffle(sampled_indices)
        return iter(sampled_indices)
    
    def __len__(self):
        return self.min_count * len(self.classes)
    
    def __min_count(self):
        return min(self.class_counts.values())
        
        

In [14]:
ds = SiameseDataset(raw_dir, transform=transforms.ToTensor())

In [ ]:
sampler = RandomUnderSampler(ds.targets)

In [16]:
dl = torch.utils.data.DataLoader(ds, sampler=sampler, batch_size=510980)

In [17]:
targets = []
indices = []
for idx in sampler:
    targets.append(ds.targets[idx])
    indices.append(idx)
print(np.bincount(targets))
print(indices[0])

[255490 255490]
87476858


In [18]:
import numpy as np

In [19]:
class_count = np.bincount(ds.targets)

In [20]:
class_count

array([87307271,   255490])

In [21]:
from collections import Counter

In [22]:
class_count = Counter(ds.targets)

In [23]:
class_count

Counter({0: 87307271, 1: 255490})

In [24]:
class_count.keys()

dict_keys([1, 0])

In [25]:
minority_class_count = min(class_count.values())
minority_class_count

255490

In [26]:
for i in class_count.items():
    print(i)

(1, 255490)
(0, 87307271)


In [27]:
print([item[0] for item in class_count.items()])

[1, 0]


In [28]:
indices = {cls: np.where(ds.targets == cls) for cls in class_count}
indices

/tmp/ipykernel_36/3412869768.py:1: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  indices = {cls: np.where(ds.targets == cls) for cls in class_count}


{1: (array([], dtype=int64),), 0: (array([], dtype=int64),)}

In [29]:
np.unique(ds.targets)

array([0, 1])

In [30]:
len(ds.targets)

87562761

In [31]:
targets = np.array(ds.targets)
targets

array([1, 0, 0, ..., 1, 0, 1])

In [32]:
np.where(targets==0)[0]

array([       1,        2,        3, ..., 87562756, 87562757, 87562759])

In [33]:
classes = np.unique(ds.targets)
classes

array([0, 1])

In [34]:
indices_per_class = {cls: np.where(targets == cls)[0] for cls in classes}
indices_per_class

{0: array([       1,        2,        3, ..., 87562756, 87562757, 87562759]),
 1: array([       0,    13233,    26465, ..., 87562755, 87562758, 87562760])}

In [35]:
sampled_indices = []

In [36]:
min_count  = min(class_count.values())
min_count

255490

In [37]:
for cls, indices in indices_per_class.items():
    sampled_indices.extend(np.random.choice(indices,min_count))

In [38]:
len(sampled_indices)

510980

In [39]:
torch.nn.Sigmoid

torch.nn.modules.activation.Sigmoid

In [40]:
from torchvision.models import mobilenetv2

In [41]:
model = mobilenetv2.MobileNetV2()

In [42]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [43]:
batch_size = 1
channels = 3
height = 224
width = 224

In [44]:
dummy_input = torch.randn(batch_size, channels, height, width)

In [45]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [46]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [47]:
dummy_input2 = model.features(dummy_input[:1])

In [48]:
dummy_input2.shape

torch.Size([1, 1280, 7, 7])

In [49]:
avgpool = torch.nn.AdaptiveAvgPool2d(1)

In [50]:
dummy_input3 = avgpool(dummy_input2)
dummy_input3.shape

torch.Size([1, 1280, 1, 1])

In [54]:
from torch import nn

In [55]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.backbone = MobileNetV2()
        self.fc = nn.Sequential(
            nn.Linear(1280,1280),
            nn.Sigmoid(),
        )

    def forward_one_branch(self, x):
        x = self.backbone.features(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1,1))
        x = torch.flatten(x)
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one_branch(input1)
        output2 = self.forward_one_branch(input2)
        return output1, output2

In [ ]:
import torch.nn.functional as F

In [57]:
from torch import nn
from torchvision.models.mobilenetv2 import MobileNetV2

In [58]:
class EuclideanDistance(nn.Module):
    def __init__(self):
        super(EuclideanDistance, self).__init__()
    
    def forward(self, output1, output2):
        return torch.sqrt(torch.sum((output1-output2)**2))


In [59]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, distance, label):
        loss_negative = (1 - label) * torch.pow(distance, 2)
        loss_positive = label * torch.pow(torch.clamp(self.margin - distance, min=0.0), 2)
        loss = torch.mean(loss_negative + loss_positive)
        return loss

In [60]:
dummy1 = torch.tensor([1,1,1,1,1,1,1])
dummy2 = torch.tensor([1,1,1,1,1,1,1])

In [61]:
loss = ContrastiveLoss()
loss(torch.tensor(0.8),torch.tensor(0))

tensor(0.6400)

In [62]:
(dummy1-dummy2)**2

tensor([0, 0, 0, 0, 0, 0, 0])

In [64]:
torch.clamp(torch.tensor(-1),torch.tensor(0))

tensor(0)

In [65]:
torch.sum(dummy1-dummy2)

tensor(0)

In [66]:
embedding_layer = Embedding()

In [67]:
output1, output2 = embedding_layer(dummy_input, dummy_input)
print(output1.shape, output2.shape)

torch.Size([1280]) torch.Size([1280])


In [68]:
torch.max(output1)

tensor(0.6795, grad_fn=<MaxBackward1>)

In [69]:
torch.max(output2)

tensor(0.6795, grad_fn=<MaxBackward1>)